In [91]:
import socket
import struct
import csv
import numpy as np
import IPython.display as disp

print "modules loaded"

spath = '/home/hadoop/ipython/vast/user/vast/'
sconnect = spath + 'lda_scores.csv'


modules loaded


In [92]:
!wc -l $sconnect
!head -n 10 $sconnect

251 /home/hadoop/ipython/vast/user/vast/lda_scores.csv
score,Date,srcIP,dstIP,SPort,DPort,Sfirst,Dfirst,Stotal,Dtotal,Spkts,Dpkts
1.50E-07,4/14/2013 15:10,10.70.68.127,172.30.0.4,6395,80,0,0,278,116,5,2
1.50E-07,4/14/2013 15:09,10.70.68.127,172.30.0.4,55759,80,19,1006,297,1114,5,2
1.51E-07,4/10/2013 17:16,10.13.77.49,172.10.0.40,61783,80,0,0,148,108,2,2
1.51E-07,4/14/2013 15:16,10.13.77.49,172.10.0.2,0,0,0,0,354,354,2,2
1.51E-07,4/14/2013 17:16,10.13.77.49,172.10.0.40,47131,80,0,0,206,162,3,3
1.51E-07,4/14/2013 14:56,10.13.77.49,172.10.0.3,35579,25,0,0,112,58,2,1
1.51E-07,4/14/2013 15:16,10.13.77.49,172.20.0.3,46032,25,0,0,496,280,8,4
1.72E-07,4/11/2013 12:57,10.200.20.2,172.30.0.4,3247,80,0,0,286,62,5,1
1.75E-07,4/14/2013 16:21,10.138.235.111,172.10.0.40,61471,80,0,0,148,108,2,2


In [30]:
#sev,tstart,srcIP,dstIP,sport,dport,proto,ipkt,ibyt,lda_score
with open(sconnect, 'r') as f:
    reader = csv.reader(f,delimiter=',') 
    reader.next();
    rowct = 1
    for row in reader:
        if int(row[0]) < 4: # 4 is the don't care
            # get src ip and dst ip
            sip = row[3]
            dip = row[4]
            # get hour and date 2014-07-08 10:10:40
            hr = row[2].split(' ')[1].split(':')[0]
            dy = row[2].split(' ')[0].split('/')[1] 
            mm = row[2].split(' ')[1].split(':')[1]

            hivestr = (" \"set hive.cli.print.header=true; SELECT parseddate,iplayerprotocol as pcode," +
            "iplayerprotocolcode as pcode, firstseensrcip as srcip,firstseendestip as dstip," +
            "firstseensrcport as sport,firstseendestport as dport,firstseensrcpayloadbytes as sfirst," +
            "firstseendestpayloadbytes as dfirst,firstseensrctotalbytes as stotal, firstseendesttotalbytes as dtotal, " +
            "firstseensrcpacketcount as  spkts, firstseendestpacketcount as  dpkts FROM vast_netflow " + 
            "WHERE ( (firstseensrcip=\'" + sip + "\' AND firstseendestip=\'" + dip + "\') OR " +
            "(firstseensrcip=\'" + dip + "\' AND firstseendestip=\'" + sip + "\') ) " +
            " AND day(parseddate)="+dy+" AND hour(parseddate)="+hr +" AND minute(parseddate)="+mm +
            " SORT BY parseddate LIMIT 100 \" > " + spath+ "edge-" + sip.replace(".","_") + "-" + 
            dip.replace(".","_") + "-" + hr + "-" + mm + ".tsv")
            disp.clear_output()
            print hr, dy, mm
            print 'processing line ',rowct
            print hivestr
            !hive -S -e $hivestr
            if rowct == 150:
                break
            rowct = rowct + 1


7 10 29
processing line  150
 "set hive.cli.print.header=true; SELECT parseddate,iplayerprotocol as pcode,iplayerprotocolcode as pcode, firstseensrcip as srcip,firstseendestip as dstip,firstseensrcport as sport,firstseendestport as dport,firstseensrcpayloadbytes as sfirst,firstseendestpayloadbytes as dfirst,firstseensrctotalbytes as stotal, firstseendesttotalbytes as dtotal, firstseensrcpacketcount as  spkts, firstseendestpacketcount as  dpkts FROM vast_netflow WHERE ( (firstseensrcip='172.10.1.119' AND firstseendestip='10.199.250.2') OR (firstseensrcip='10.199.250.2' AND firstseendestip='172.10.1.119') )  AND day(parseddate)=10 AND hour(parseddate)=7 AND minute(parseddate)=29 SORT BY parseddate LIMIT 100 " > /home/hadoop/ipython/vast/user/vast/edge-172_10_1_119-10_199_250_2-7-29.tsv
15/04/08 02:02:12 WARN conf.HiveConf: DEPRECATED: Configuration property hive.metastore.local no longer has any effect. Make sure to provide a valid value for hive.metastore.uris if you are connecting to a

In [93]:
#hive query for chord diagram tsvs: TODO
!ls $spath/chord*.tsv
!cat $spath/chord-10_0_0_9.tsv


/home/hadoop/ipython/vast/user/vast//chord-10_0_0_10.tsv
/home/hadoop/ipython/vast/user/vast//chord-10_0_0_12.tsv
/home/hadoop/ipython/vast/user/vast//chord-10_0_0_6.tsv
/home/hadoop/ipython/vast/user/vast//chord-10_0_0_7.tsv
/home/hadoop/ipython/vast/user/vast//chord-10_0_0_9.tsv
/home/hadoop/ipython/vast/user/vast//chord-10_0_3_77.tsv
/home/hadoop/ipython/vast/user/vast//chord-10_12_14_15.tsv
/home/hadoop/ipython/vast/user/vast//chord-10_12_15_152.tsv
/home/hadoop/ipython/vast/user/vast//chord-10_13_77_49.tsv
/home/hadoop/ipython/vast/user/vast//chord-10_138_235_111.tsv
/home/hadoop/ipython/vast/user/vast//chord-10_17_15_10.tsv
/home/hadoop/ipython/vast/user/vast//chord-10_199_250_2.tsv
/home/hadoop/ipython/vast/user/vast//chord-10_6_6_7.tsv
/home/hadoop/ipython/vast/user/vast//chord-172_0_0_1.tsv
/home/hadoop/ipython/vast/user/vast//chord-172_10_0_2.tsv
/home/hadoop/ipython/vast/user/vast//chord-172_10_0_3.tsv
/home/hadoop/ipython/vast/user/vast//chord-172_10_0_40.tsv
/home/hadoop/i

In [94]:
chordfiles = ['/home/hadoop/ipython/vast/user/vast/chord-10_0_0_10.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-10_0_0_12.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-10_0_0_6.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-10_0_0_7.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-10_0_0_9.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-10_0_3_77.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-10_12_14_15.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-10_12_15_152.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-10_13_77_49.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-10_138_235_111.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-10_17_15_10.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-10_199_250_2.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-10_6_6_7.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-172_0_0_1.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-172_10_0_2.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-172_10_0_3.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-172_10_0_40.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-172_10_0_4.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-172_10_0_5.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-172_10_0_7.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-172_10_0_8.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-172_10_0_9.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-172_20_0_15.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-172_20_0_2.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-172_20_0_3.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-172_20_0_4.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-172_20_0_5.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-172_20_0_6.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-172_30_0_3.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-172_30_0_4.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-172_30_0_6.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-172_30_0_7.tsv',
'/home/hadoop/ipython/vast/user/vast/chord-172_30_1_218.tsv']

In [95]:
#ceate chord matrix
for chordfile in chordfiles:
    #chordfile = 'chord-10_0_0_9'
    ll = 30;
    #load csv into array
    #with open(spath+chordfile+'.tsv', 'rb') as f:
    with open(chordfile, 'rb') as f:
        reader = csv.reader(f,delimiter='\t') 
        reader.next()
        carry = np.zeros([ll,3],np.int64)
        rowct = 0
        for row in reader:
            #print row
            carry[rowct,0] = struct.unpack("!L", socket.inet_aton(row[0]))[0]    
            carry[rowct,1] = struct.unpack("!L", socket.inet_aton(row[1]))[0]    
            carry[rowct,2] = np.int64(row[2])
            rowct += 1
            if rowct == ll-1:
                break

    dim = np.zeros(2*ll,np.int64)
    dim = np.concatenate([carry[:,0],carry[:,1]],axis=0)
    dim = np.unique(dim)
    chordm = np.zeros([len(dim),len(dim)],np.int64)
    for j in carry:
        xcoord = np.where(dim == j[0]);
        ycoord = np.where(dim == j[1]);
        chordm[xcoord,ycoord] = j[2]
    print 'chord data loaded.'
    dimstr = []
    for j in dim:
        dimstr.append(socket.inet_ntoa(struct.pack('!L', j)))
    headr = ",".join(dimstr)
    fmtr = r'%1d ' * len(dim)
    #with open(spath+chordfile+'.csv', 'wb') as f:
    with open(chordfile.replace('.tsv','.csv'), 'wb') as f:
        riter = csv.writer(f,delimiter=',')
        # header line?
        for j in chordm:
            #outstr = '['+','.join(map(str, j))+'],'
            riter.writerow(j)
    #np.savetxt(g,chordm,delimiter=',',fmt=fmtr,header=headr)


chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.
chord data loaded.


In [80]:
!cat $spath/chord-10_0_0_9.csv